In [ ]:
import pandas as pd
pd.__version__
import matplotlib.pyplot as plt
%matplotlib inline



In [ ]:
import os
import pandas as pd
import chardet
import pickle

def unir_csv_en_carpeta(carpeta, extension='.csv'):
    """Unir todos los archivos csv de una carpeta en un solo dataframe."""
    lista_dataframes = []
    for archivo in os.listdir(carpeta):
        # Si el archivo tiene la extensión que buscamos
        if archivo.endswith(extension):
            print(archivo)
            ruta_archivo = os.path.join(carpeta, archivo)
            # Detectar el encoding del archivo
            with open(ruta_archivo, 'rb') as f:
                result = chardet.detect(f.read())
            # Leer el archivo con el encoding detectado
            df = pd.read_csv(ruta_archivo, encoding=result['encoding'], low_memory=False)
            # incluir columna con el nombre del archivo
            df['archivo'] = archivo
            lista_dataframes.append(df)
    df_unido = pd.concat(lista_dataframes, ignore_index=True)
    return df_unido

In [ ]:
inscritos = unir_csv_en_carpeta('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/inscritos')
# guarda el dataframe en un pickle
inscritos.to_pickle('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/inscritos.pkl')

In [ ]:
postulaciones = unir_csv_en_carpeta('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/postulaciones')
postulaciones.to_pickle('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/postulaciones.pkl')

In [ ]:
acepta_asigna = unir_csv_en_carpeta('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/acepta_asigna')
acepta_asigna.to_pickle('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/acepta_asigna.pkl')

In [ ]:
# Funcion que devuelve un listado y un dataframe que Elimina todas las columnas que tengan más del 50% de valores faltantes en el dataframe postulaciones

def eliminar_vacios(df):
    threshold = len(df) * 0.5 # definir el umbral como el 50% de los valores en el DataFrame
    dropped_cols = list(df.columns[df.isna().mean() > 0.5]) # crear una lista de columnas eliminadas
    df = df.dropna(thresh=threshold, axis=1)
    return dropped_cols, df

# aplica a inscritos
dropped_cols_inscritos, inscritos = eliminar_vacios(inscritos)
print("Columnas eliminadas:")
print(dropped_cols_inscritos)

# aplica a postulaciones
dropped_cols_postulaciones, postulaciones = eliminar_vacios(postulaciones)
print("Columnas eliminadas:")
print(dropped_cols_postulaciones)

# aplica a acepta_asigna
dropped_cols_acepta_asigna, acepta_asigna = eliminar_vacios(acepta_asigna)
print("Columnas eliminadas:")
print(dropped_cols_acepta_asigna)

In [ ]:
# Realiza un reporte EDA (Exploratory Data Analysis) de acepta_asigna con pandas profiling, debes seguir los siguientes pasos
# 1. Instalar pandas profiling
# 2. Importar pandas profiling

import pandas_profiling

# 3. Generar el reporte
reporte = postulaciones.profile_report()

# 4. Guardar el reporte en un archivo html
reporte.to_file(output_file='reporte_postulaciones.html')

# 3. Generar el reporte
reporte = acepta_asigna.profile_report()

# 4. Guardar el reporte en un archivo html
reporte.to_file(output_file='reporte_acepta_asigna.html')

# 3. Generar el reporte
reporte = inscritos.profile_report()

# 4. Guardar el reporte en un archivo html
reporte.to_file(output_file='reporte_inscritos.html')


In [ ]:
acepta_asigna.to_pickle('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/acepta_asigna.pkl')
postulaciones.to_pickle('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/postulaciones.pkl')
inscritos.to_pickle('/Users/julianmaya/Downloads/senescyt/BASES_ACCESO_ANONIMIZADAS/inscritos.pkl')

# Separar en multiples pickles para poder subir a github

In [75]:
# Funcion que divide el pickle inscritos en n partes
def dividir_pickle(pickle, n, nombre):
    df = pd.read_pickle(pickle)
    df_partes = []
    for i in range(n):
        df_partes.append(df.iloc[i::n])
    for i in range(len(df_partes)):
        df_partes[i].to_pickle(nombre + str(i) + '.pkl')

In [92]:
dividir_pickle('acepta_asigna.pkl',6,'acepta_asigna_')

# Unir los multiples pickles de una carpeta

In [90]:
# Funcion que une los archivos pickle en un solo dataframe
def unir_pickle(nombre, n):
    lista_dataframes = []
    for i in range(n):
        df = pd.read_pickle(nombre + str(i) + '.pkl')
        lista_dataframes.append(df)
    df_unido = pd.concat(lista_dataframes, ignore_index=True)
    return df_unido

In [95]:
test_inscritos = unir_pickle('pickles_partes/inscritos_',7)
test_inscritos

,Unnamed: 0,USU_FECHAREGISTRO,USU_ESTADO,USU_ESTADO_CIVIL,USU_FECHA_NAC,INS_SEXO,INS_ID,INS_FECHA,INS_ESTADO,INS_PASO,...,USU_NACIONALIDAD,USU_NACIONALIDAD_EXTRANJERA,IAS_OTRA_DISCAPACIDAD,INS_TIPO,CARGA_ENCUESTA,FINALIZA_INSCRIPCION,OTRA_DISCAPACIDAD,TITULO_HOMOLOGADO,INTERNET_DOMICILIO,COMPUTADORA_DOMICILIO
0,1,4/5/2018 13:37:49,A,S,28/9/2000 0:00:00,MUJER,7433157.0,7/5/2019 19:29:18,TERMINADO,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,30/4/2019 21:08:10,A,D,6/12/1956 0:00:00,HOMBRE,7333209.0,30/4/2019 21:11:55,TERMINADO,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,15,3/5/2019 10:27:54,A,V,24/5/1960 0:00:00,MUJER,7423673.0,3/5/2019 10:51:07,TERMINADO,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22,6/5/2019 23:42:50,A,D,2/6/1968 0:00:00,HOMBRE,7509166.0,6/5/2019 23:47:35,TERMINADO,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,29,7/5/2019 7:58:21,A,S,5/3/1975 0:00:00,HOMBRE,7511159.0,7/5/2019 8:09:50,TERMINADO,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1393505,213945,44040.82719,A,S,35899.0,MUJER,NaN,44040.82848,TERMINADO,5,...,ECUATORIANA,1.0,0.0,I,SI,EFA Y REGISTRO COMPLETO,NO,NO,SI,SI
1393506,213952,35360.79167,A,C,35360.0,MUJER,NaN,44041.62396,TERMINADO,5,...,COLOMBIANA,88.0,0.0,I,SI,EFA Y REGISTRO COMPLETO,NO,NO,SI,SI
1393507,213959,44041.74995,A,C,36783.0,HOMBRE,NaN,44041.75249,TERMINADO,5,...,COLOMBIANA,88.0,0.0,I,SI,EFA Y REGISTRO COMPLETO,NO,NO,SI,SI
1393508,213966,36251.79167,A,NaN,36251.0,MUJER,NaN,44043.86126,TERMINADO,5,...,COLOMBIANA,88.0,0.0,I,NO,EFA Y REGISTRO INCOMPLETO,NO,NO,SI,SI


: 

In [86]:
test_postulaciones = unir_pickle('pickles_partes/postulaciones_',36)
test_postulaciones

In [94]:
test_acepta_asigna = unir_pickle('pickles_partes/acepta_asigna_',6)
test_acepta_asigna

,Unnamed: 0,ASA_ID,INI_ID,INS_ID,GENERO,USU_FECHA_NAC,USU_NACIONALIDAD,ETNIA,COD_PARROQUIA_RESIDE,PARROQUIA_RESIDE,...,PRD_ID_NUM_ASIGNACION,INSTANCIA_POSTULACION,INSTANCIA_ASIGNACION,ASA_OBSERVACION,ASA_GRATUIDAD,GRATUIDAD,ACEPTA_CUPO_DDA,ASA_ESTADO_DDA,cod_final,archivo
0,1,2242680.0,6835166.0,11425152.0,FEMENINO,23233.0,ECUATORIANA,Mestizo/a,170109.0,CHIMBACALLE,...,446.0,PRIMERA POSTULACION,PRIMERA ASIGNACIÓN DE CUPOS,1era Asignación de cupos 1era Postulación P22 ...,0.0,MANTIENE GRATUIDAD,0.0,1.0,2702700138,asigna-acepta_per22.csv
1,7,2191225.0,6831573.0,11417986.0,FEMENINO,29060.0,ECUATORIANA,Mestizo/a,110102.0,SAN SEBASTIÁN,...,446.0,PRIMERA POSTULACION,PRIMERA ASIGNACIÓN DE CUPOS,1era Asignación de cupos 1era Postulación P22 ...,0.0,MANTIENE GRATUIDAD,1.0,1.0,2714170192,asigna-acepta_per22.csv
2,13,2253067.0,6915731.0,11586274.0,FEMENINO,26628.0,ECUATORIANA,Mestizo/a,30450.0,LA TRONCAL,...,446.0,PRIMERA POSTULACION,PRIMERA ASIGNACIÓN DE CUPOS,1era Asignación de cupos 1era Postulación P22 ...,0.0,MANTIENE GRATUIDAD,1.0,1.0,2730120110,asigna-acepta_per22.csv
3,19,2219409.0,7070439.0,11895754.0,FEMENINO,26751.0,ECUATORIANA,Mestizo/a,10170.0,VALLE,...,446.0,PRIMERA POSTULACION,PRIMERA ASIGNACIÓN DE CUPOS,1era Asignación de cupos 1era Postulación P22 ...,0.0,MANTIENE GRATUIDAD,1.0,1.0,2761220101,asigna-acepta_per22.csv
4,25,2236187.0,6993412.0,11741684.0,FEMENINO,27133.0,ECUATORIANA,Mestizo/a,10101.0,BELLAVISTA,...,446.0,PRIMERA POSTULACION,PRIMERA ASIGNACIÓN DE CUPOS,1era Asignación de cupos 1era Postulación P22 ...,0.0,MANTIENE GRATUIDAD,1.0,1.0,2741230110,asigna-acepta_per22.csv
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607459,112013,NaN,4052688.0,7229693.0,FEMENINO,36267,COLOMBIA,Mestizo,40101.0,GONZ√ÅLEZ SU√ÅREZ,...,531.0,SEGUNDA POSTULACI√ìN,CUARTA ASIGNACI√ìN DE CUPOS,4ta Asignaci√≥n de cupos 2da Postulaci√≥n P18 ...,NaN,NaN,NaN,NaN,2230786065,asigna-acepta_per18.csv
607460,112019,NaN,4614648.0,7992237.0,MASCULINO,33912,COLOMBIA,Mestizo,40102.0,TULC√ÅN,...,492.0,SEGUNDA POSTULACI√ìN,TERCERA ASIGNACI√ìN DE CUPOS,2da Asignaci√≥n de cupos 2da Postulaci√≥n P18 ...,NaN,NaN,NaN,NaN,2039256101,asigna-acepta_per18.csv
607461,112025,NaN,4140331.0,7404971.0,FEMENINO,36349,COLOMBIA,Mestizo,40102.0,TULC√ÅN,...,492.0,SEGUNDA POSTULACI√ìN,TERCERA ASIGNACI√ìN DE CUPOS,2da Asignaci√≥n de cupos 2da Postulaci√≥n P18 ...,NaN,NaN,NaN,NaN,2253286383,asigna-acepta_per18.csv
607462,112031,NaN,4128821.0,7381991.0,FEMENINO,36385,CUBA,Blanco,100302.0,SAN FRANCISCO,...,494.0,TERCERA POSTULACION,QUINTA ASIGNACI√ìN DE CUPOS,5ta Asignaci√≥n de cupos 3ra Postulaci√≥n P18 ...,NaN,NaN,NaN,NaN,2064586283,asigna-acepta_per18.csv
